In [1]:
# CAGED (Cadastro Geral de Empregados e Desempregados)
# name:         caged_xlsx_to_csv
# date:         29/03/2022 - 01/04/2022
# description:  caged xlsx file data processed, optimized and saved into csv file

import pandas as pd


In [2]:
# creates panda database with all file sheets               - dfb
# creates panda dataframe with specific sheets              - first_df
# creates /path/files names for getting and saving data     - file_in, file_out

file_in = '3-tabelas_DEZ.xlsx'
file_out = 'caged.csv'

skiprows = 4  # [0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0]

dfb = pd.read_excel(file_in, sheet_name=None, skiprows=skiprows)
first_df = dfb['Tabela 5.1']
# df


/home/noobgoodie/.local/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/noobgoodie/.local/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/noobgoodie/.local/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [3]:
# optimizes data mensal evolution by dates with Saldos

months_names_br = ('Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho',
                   'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro')
month_names_dec = ('01',      '02',        '03',    '04',    '05',
                   '06',    '07',    '08',     '09',       '10',      '11',       '12')

# prepare dict variable to the replacements
mbr2mdec = dict(zip(months_names_br, month_names_dec))

df2 = first_df.copy()

# selecting date cells only
df2 = df2[(df2['Mês'].str.slice(-4) >= '1900') &
          (df2['Mês'].str.slice(-4) <= '2099')]

# months name to months numbers replacements
df2['Mês'] = df2['Mês'].replace(mbr2mdec, regex=True)
df2['Mês'] = df2['Mês'].str.replace('/', '-')
df2['Mês'] = pd.to_datetime(
    df2['Mês'], format='%Y-%m-%d', infer_datetime_format=True)


def try_int(val):
    try:
        val = float(val)
        val = int(val)
        return True
    except:
        return False


filter = df2['Saldos'].apply(try_int)
df2.drop(df2[filter == False].index, inplace=True)

df2['Saldos'] = df2['Saldos'].astype(int)
first_out_df = df2[['Mês', 'Saldos']]
first_out_df


,Mês,Saldos
0,2020-01-01,112211
1,2020-02-01,217496
2,2020-03-01,-294777
3,2020-04-01,-981805
4,2020-05-01,-398217
5,2020-06-01,-53505
6,2020-07-01,108763
7,2020-08-01,214809
8,2020-09-01,299497
9,2020-10-01,366166


In [4]:
# creates panda dataframe with specific sheets  - second_df


second_df = pd.read_excel(file_in, sheet_name=11, skiprows=4, header=[0, 1])


In [5]:
# optimizes data mensal evolution by dates and regions with Saldos

regions_table = ('Região e UF', 'Unnamed: 1_level_1')
regions_names = ('Norte', 'Nordeste', 'Sudeste',
                 'Sul', 'Centro-Oeste', 'Não identificado')

# copying df for reutilizing
df2 = second_df.copy()


# function to create a dataframe columns list to use
def createColumnsList(param_df):
    columns_list = []
    column_names = param_df.columns.values.tolist()
    for column_name in column_names:
        # column_name has 2 parts (date, info_type)
        col_date, col_info = column_name
        month = col_date.split('/')[0]
        if month in months_names_br:
            if col_info.lower().find('saldos') != -1:
                columns_list.append(column_name)

    return(columns_list)


months_list = createColumnsList(df2)

# TRANSFORMAR EM FUNCAO
for month in months_list:
##for regions_list_index in range(0, 2):
    print('-MES----------------------------------------------------')
    print(month)
    print('-FILTRO----------------------------------------------------')
    filter = df2[month].apply(try_int)
    print(filter)
    df_tmp = df2[month]
    df_tmp.transpose
    print('-COLUNA_SALDO----------------------------------------------')
    print(df_tmp)
    print('-VALORES_INVALIDOS-----------------------------------------')
    print( df_tmp[filter == False] )
    print('-COLUNA ARRUMADA-------------------------------------------')
    df2.drop(df_tmp[filter == False].index, inplace=True)
    print(df2[[month]])    

df2[months_list]

# selecting specifics regions
df3 = df2[df2[regions_table].isin(regions_names)]
second_out_df = df3[months_list]

second_out_df = df3[months_list].astype(int)
second_out_df = second_out_df.transpose()

second_out_df


-MES----------------------------------------------------
('Janeiro/2020', 'Saldos')
-FILTRO----------------------------------------------------
0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
30     True
31     True
32     True
33     True
34    False
35    False
36    False
37    False
38    False
39    False
Name: (Janeiro/2020, Saldos), dtype: bool
-COLUNA_SALDO----------------------------------------------
0     112211.0
1       2818.0
2        -76.0
3        137.0
4        928.0
5        380.0
6       1178.0
7         74.0
8        197.0
9      -3339.0
10       427.0
11        31.0
12      2417.0
13     -1033.0
14     -2567.0
15      -585.0
16     -5046.0
17       178.0
18  

,
Janeiro/2020,Saldos
Fevereiro/2020,Saldos
Março/2020,Saldos
Abril/2020,Saldos
Maio/2020,Saldos
Junho/2020,Saldos
Julho/2020,Saldos
Agosto/2020,Saldos
Setembro/2020,Saldos
Outubro/2020,Saldos


In [6]:
# creates panda dataframe with specific sheets  - third_df

third_df = pd.read_excel(file_in, sheet_name=9, skiprows=4, header=[0, 1])


In [7]:
# optimizes data mensal evolution by dates and sectors with Saldos

gr_act_econ_table = (
    'Grupamento de Atividades Econômicas e Seção CNAE 2.0', 'Unnamed: 1_level_1')
sector_names = ('Agricultura, pecuária, produção florestal, pesca e aquicultura', 'Indústria geral', 'Construção',
                'Comércio; reparação de veículos automotores e motocicletas', 'Serviços', 'Não identificado***')

# copying df for reutilizing
df2 = third_df.copy()

sectors_list = createColumnsList(df2)

for month in sectors_list:
    print('-MES----------------------------------------------------')
    print(month)
    print('-FILTRO----------------------------------------------------')
    filter = df2[month].apply(try_int)
    print(filter)
    df_tmp = df2[month]
    df_tmp.transpose
    print('-COLUNA_SALDO----------------------------------------------')
    print(df_tmp)
    print('-VALORES_INVALIDOS-----------------------------------------')
    print( df_tmp[filter == False] )
    print('-COLUNA ARRUMADA-------------------------------------------')
    df2.drop(df_tmp[filter == False].index, inplace=True)
    print(df2[[month]])    

df2[sectors_list]


# selecting specifics sectors
df3 = df2[df2[gr_act_econ_table].isin(sector_names)]
third_out_df = df3[sectors_list]

third_out_df = df3[sectors_list].astype(int)
third_out_df = third_out_df.transpose()
third_out_df


-MES----------------------------------------------------
('Janeiro/2020', 'Saldos')
-FILTRO----------------------------------------------------
0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28    False
29    False
30    False
31    False
32    False
33    False
34    False
Name: (Janeiro/2020, Saldos), dtype: bool
-COLUNA_SALDO----------------------------------------------
0     112211.0
1      16500.0
2      58288.0
3         45.0
4      57901.0
5       -387.0
6        729.0
7      35909.0
8     -51675.0
9      53189.0
10     -1920.0
11      4399.0
12     29500.0
13      5128.0
14      1747.0
15      1227.0
16     10455.0
17     10943.0
18     14190.0
19      -505.0
20      8590.0
21      6105.0
22  

,
Janeiro/2020,Saldos
Fevereiro/2020,Saldos
Março/2020,Saldos
Abril/2020,Saldos
Maio/2020,Saldos
Junho/2020,Saldos
Julho/2020,Saldos
Agosto/2020,Saldos
Setembro/2020,Saldos
Outubro/2020,Saldos


In [8]:
# creates csv file

first_columns_names_out = {'Mês': '', 'Saldos': 'total'}
second_columns_names_out = {1: 'reg1', 9: 'reg2', 19: 'reg3', 24: 'reg4', 28: 'reg5', 33: 'reg9'}
third_columns_names_out = {1: 'setA', 2: 'setE', 7: 'setF', 8: 'setG', 9: 'setU', 27: 'setZ'}

# remove index and columns
temp_first_out_df = first_out_df.reset_index(drop=True)
temp_first_out_df.rename(columns=first_columns_names_out, inplace=True)

temp_second_out_df = second_out_df.reset_index(drop=True)
temp_second_out_df.rename(columns=second_columns_names_out, inplace=True)

temp_third_out_df = third_out_df.reset_index(drop=True)
temp_third_out_df.rename(columns=third_columns_names_out, inplace=True)

final_out_df = pd.concat([temp_first_out_df, temp_second_out_df, temp_third_out_df], axis=1)
final_out_df.to_csv(file_out, index=False)
final_out_df


,,total
0,2020-01-01,112211.0
1,2020-02-01,217496.0
2,2020-03-01,-294777.0
3,2020-04-01,-981805.0
4,2020-05-01,-398217.0
5,2020-06-01,-53505.0
6,2020-07-01,108763.0
7,2020-08-01,214809.0
8,2020-09-01,299497.0
9,2020-10-01,366166.0


In [9]:
f_in = 'caged_2022_02.csv'
myf_in = 'caged.csv'

csv_df = pd.read_csv(f_in)
mycsv_df = pd.read_csv(myf_in)
csv_df.rename(columns={'Unnamed: 0':''}, inplace=True)
mycsv_df.rename(columns={'Unnamed: 0':''}, inplace=True)

#rdf = csv_df[list(mycsv_df.columns)]
#rdf
#mycsv_df == rdf

